In [1]:
import pymysql as mariadb

try:
    conn = mariadb.connect(
     host='db',
     user='root',
     passwd='1234',
     db='rental',
     charset='utf8mb4'
    )
    
    cursor = conn.cursor()
    sql = """
        CREATE TABLE `negative` (
  `ID` INT NOT NULL AUTO_INCREMENT,
  `name` VARCHAR(100) NULL,
  `address` VARCHAR(200) NULL,
  `typeID` INT NULL,
  `lat` DECIMAL(20,17) NULL,
  `lng` DECIMAL(20,17) NULL,
  `cityID` INT NULL,
  PRIMARY KEY (`ID`))
  ENGINE = InnoDB DEFAULT  
  CHARACTER SET = utf8mb4 COLLATE = utf8mb4_general_ci;
        """
    try:
        cursor.execute(sql)
    except Exception as e:
        print('Execute Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))
        cursor.close()
        conn.close()
except Exception as e:
    print('Connection Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))

In [2]:
import glob
from multiprocessing import Process, Queue
import json
q1=Queue()
fil = glob.glob('negative/*')
for file in fil:
    for detail in glob.glob('{}/*'.format(file)):
        with open('{}'.format(detail),encoding='utf-8') as f:
            jSon = json.load(f)
            #print(jSon)
            q1.put(jSon)

print(q1.qsize())

6069


In [3]:
def write(obj):
    obj = obj
    sql = '''INSERT INTO negative (name,address,typeID,lat,lng,cityID) VALUES 
                (%s,%s,%s,%s,%s,%s)'''
    values = ( obj['name'], obj['address'],obj['typeID'], obj['lat'],obj['lng'], obj['cityID'])
    try:
        cursor.execute(sql,values)
    except Exception as e:
            print(obj)
            print('Execute Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))

In [4]:
import pymysql as mariadb
import time
start = time.time()
conn = mariadb.connect(
            host='db',
            user='root',
            passwd='1234',
            db='rental',
            charset='utf8mb4'
        )
cursor = conn.cursor()
while not q1.empty():
    try:
        obj = q1.get()
#         print(obj)
        write(obj)
    except Exception as e:
        print('Execute Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))
        print(obj)
conn.commit()
print('commit')
conn.close()
    
print('total time = {}'.format(time.time()-start))

commit
total time = 1.3467416763305664
